In [1]:
import affine as af

/Users/const/AF/affine/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
af.trace()

In [4]:
async for result in af.dataset(tail=100):
    print(result.challenge.env, result.evaluation.score)


n=3 k=2 m=12 1.0
 1.0
 1.0
 1.0
n=3 k=2 m=12 1.0
n=3 k=2 m=12 1.0
 0.8165680473372781
 0.0
 0.0
 1.0
n=3 k=2 m=12 1.0
 1.0
 1.0
n=3 k=2 m=12 0.0
 0.0
n=3 k=2 m=12 0.0
 0.0
 0.0
 1.0
n=3 k=2 m=12 1.0
 0.23529411764705882
n=3 k=2 m=12 1.0
 1.0
 1.0
n=3 k=2 m=12 0.0
 0.0
 0.0
 1.0
 1.0
 1.0
n=3 k=2 m=12 1.0
n=3 k=2 m=12 1.0
n=3 k=2 m=12 1.0
 0.0
 0.0
 0.0
n=3 k=2 m=12 1.0
 1.0
 1.0
n=3 k=2 m=12 0.0
 0.0
 0.0
n=3 k=2 m=12 0.0
 0.0
 0.0
 1.0
 1.0
 1.0
n=3 k=2 m=12 1.0
n=3 k=2 m=12 1.0
 1.0
n=3 k=2 m=12 1.0
 0.0
 0.0
n=3 k=2 m=12 0.0
 0.0
 0.0
n=3 k=2 m=12 1.0
 0.0
 0.0
 0.0
 0.0
n=3 k=2 m=12 0.0
